# Sysconfig: управление конфигурацией интерпретатора во время компиляции

Средства, входящие в модуль __sysconfig__, были перенесены из модуля __distutils__ c целью выделения их в виде независимого модуля. Этот модуль включает функции, которые определяют настройки, используемые для компиляции и установки текущего интерпретатора.

## Конфигурационные переменные

Доступ к конфигурационным параметрам времени сборки предоставляют две функции: __get_config_vars()__ и __get_config_var()__. Функция __get_config_vars()__ возвращает словарь, сопоставляющий имена конфигурационных переменных c их значениями.

In [2]:
import sysconfig

config_values = sysconfig.get_config_vars()
print('Found {} configuration settings'.format(len(config_values.keys())))

print('\nSome highlights:\n')

print(' Installation prefixes:')
print(' prefix={prefix}'.format(**config_values))
print(' exec_prefix={exec_prefix}'.format(**config_values))
      
print('\n Version info:')
print(' py_version={py_version}'.format(**config_values))
print(' py_version_short={py_version_short}'.format(**config_values))
print(' py_version_nodot={py_version_nodot}'.format(**config_values))
      
print('\n Base directories:')
print(' base={base}'.format(**config_values))
print(' platbase={platbase}'.format(**config_values))
print(' userbase={userbase}'.format(**config_values))
print(' srcdir={srcdir}'.format(**config_values))

print('\n Compiler and linker flags:')
print(' LDFLAGS={LDFLAGS}'.format(**config_values))
print(' BASECFLAGS={BASECFLAGS}'.format(**config_values))
print(' Py_ENABLE_SHARED={Py_ENABLE_SHARED}'.format(**config_values))

Found 21 configuration settings

Some highlights:

 Installation prefixes:
 prefix=C:\Users\esiki\anaconda3
 exec_prefix=C:\Users\esiki\anaconda3

 Version info:
 py_version=3.8.3
 py_version_short=3.8
 py_version_nodot=38

 Base directories:
 base=C:\Users\esiki\anaconda3
 platbase=C:\Users\esiki\anaconda3
 userbase=C:\Users\esiki\AppData\Roaming\Python
 srcdir=C:\Users\esiki\anaconda3

 Compiler and linker flags:


KeyError: 'LDFLAGS'

Уровень детализации, доступный через API __sysconfig__, зависит от платформы, на которой выполняется программа. В случае __POSIX-систем__, таких как __Linux__ и __OS X__, выполняется анализ файла __makefile__, используемого для сборки интерпретатора, и заголовочного файла __config.h__, сгенерированного для сборки, и все переменные, обнаруженные в каждом файле, станоятся доступными. В случае таких систем, как __Windows__, которые не являются __POSIX-системами__, состав параметров ограничивается несколькими путями, расширениями имен файлов и информацией о версии.

В результате передачи имен переменных функции __get_config_vars()__ возвращаемое значение превращается в список, который создается путем присоединения значений всех этих переменных.

In [18]:
import sysconfig

bases = sysconfig.get_config_vars('base', 'platbase', 'userbase')
print('Base directories:')
for b in bases:
    print(' ', b)

Base directories:
  C:\Users\esiki\anaconda3
  C:\Users\esiki\anaconda3
  C:\Users\esiki\AppData\Roaming\Python


Если необходимо значение лишь одного конфигурационного параметра, его можно получить c помощью функции __get_config_var()__.

In [19]:
import sysconfig
print('User base directory:', sysconfig.get_config_var('userbase'))
print('Unknown variable :', sysconfig.get_config_var('NoSuchVariable'))

User base directory: C:\Users\esiki\AppData\Roaming\Python
Unknown variable : None


Если найти переменную не удается, функция __get_config_var()__ не возбуждает исключение, а возвращает значение __None__.

## Пути к каталогам установки

Модуль __sysconfig__ в основном предназначен для использования инструментами установки и создания пакетов. Как следствие, хоть он и предоставляет доступ к общим параметрам конфигурации, таким как версия интерпретатора, он фокусируется на информации, необходимой для определения местонахождения компонентов дистрибутива Python, установленных в данный момент в системе.
Местоположения, используемые для установки пакета, зависят от используемой схемы.


Схема — это набор платформозависимых каталогов, заданных но умолчанию и организованных в соответствии со стандартами и рекомендациями по созданию пакетов для данной платформы. Для установки в расположения, доступные всему сайту, или в личные каталоги, принадлежащие конкретным пользователям, используются различные схемы. Полный набор схем можно получить c помощью
функции __get_scheme_names()__.

In [1]:
import sysconfig

for name in sysconfig.get_scheme_names():
    print(name)

nt
nt_user
osx_framework_user
posix_home
posix_prefix
posix_user


Понятия “текущая схема” как такового не существует. Вместо этого используемая по умолчанию схема зависит от опций, предоставленных программе установки. Если текущая система выполняется под управлением POSIX-совместимой операционной системы, то по умолчанию используется схема __posix_prefix__. В противном случае схемой по умолчанию является имя операционной системы, определенное в атрибуте __os.name__.


Каждая схема определяет набор путей, используемых для установки пакетов. Для получения списка имен этих путей следует использовать функцию __get_path_names()__.

In [2]:
import sysconfig

for name in sysconfig.get_path_names():
    print(name)

stdlib
platstdlib
purelib
platlib
include
scripts
data


__stdlib__ - Файлы стандартной библиотеки Python, не зависящие от платформы


__platstdlib__ - Файлы стандартной библиотеки Python, зависящие от платформы


__platlib__ - Файлы, зависящие от сайта и платформы


__purelib__ - Файлы, зависящие от сайта, но не зависящие от платформы


__include__ - Заголовочные файлы, не зависящие от платформы


__platinclude__ - Заголовочные файлы, зависящие от платформы


__scripts__ - Исполняемые файлы сценариев


__data__ - Файлы данных

Для получения отдельного пути следует использовать функцию __get_path()__.

In [7]:
import sysconfig
import pprint

for scheme in ['posix_prefix', 'posix_user']:
    print(scheme)
    print('=' * len(scheme))
    print('purelib = ', sysconfig.get_path(name='purelib', scheme=scheme))

    print()

posix_prefix
purelib =  C:\Users\esiki\anaconda3\lib\python3.8\site-packages

posix_user
purelib =  C:\Users\esiki\AppData\Roaming\Python\lib\python3.8\site-packages



Данный пример демонстрирует различия между путями, действующими в рамках всей системы и используемыми для схемы __posix_prefix__, и специфическими для пользователя значениями для схемы __posix_user__.

## Информация о версии и платформе Python

В то время как модуль __sys__ включает некоторые базовые средства идентификации платформы, он недостаточно специфичен для того, чтобы его можно было использовать для установки двоичных пакетов, поскольку атрибут __sys.platform__ не всегда включает информацию об
архитектуре оборудования, размерах инструкций и других значениях, которые влияют на совместимость двоичных библиотек. Для получения более точного спецификатора платформы следует использовать функцию __get_platform()__.

In [8]:
import sysconfig

print(sysconfig.get_platform())

win-amd64


Для удобства версия интерпретатора из атрибута __sys.version_info__ также доступна через функцию __get_python_version()__ в модуле __sysconfig__.

In [9]:
import sysconfig
import sys

print('sysconfig.get_python_version():', sysconfig.get_python_version())
print('\nsys.version_info:')
print(' major :', sys.version_info.major)
print(' minor :', sys.version_info.minor)
print(' micro :', sys.version_info.micro)
print(' releaselevel:', sys.version_info.releaselevel)
print(' serial :', sys.version_info.serial)

sysconfig.get_python_version(): 3.8

sys.version_info:
 major : 3
 minor : 8
 micro : 3
 releaselevel: final
 serial : 0


Функция __get_python_version()__ возвращает строку, которую можно использовать для создания пути, специфического для версии.